In [2]:
import json
import pandas as pd
import ast
import datetime

In [251]:
def convert_unixtime(utime):
    return (datetime.datetime.fromtimestamp(int(utime)).strftime('%d-%m'))

def convert_country_code(code):
    mapping = {"CO":"Colombia", "BR": "Brazil", "VE": "Venezuela", "PE": "Peru", "UY": "Uruguay", 
               "PY": "Paraguay", "PA": "Panama", "CL": "Chile", "EC":"Ecuador", "AR": "Argentina",
               "CR": "Costa Rica", "GY": "Guyana", "MX": "Mexico", "US": "United States",  
              }
    return mapping[code]

def cut(df, col, values, savedcols= ['Location', 'dau_audience', 'mau_audience']):
    df_slice = {}
    
    for i, v in enumerate(values):
        df_slice[i] = df[df[col] == v][savedcols]
    
    merge = pd.merge(df_slice[0], df_slice[1], on=["Location"], suffixes=("_%s" % (values[0]), "_%s" % (values[1])))
    for i in range(2, len(values)):
        #print("Adding suffix for:" + values[i])
        merge = pd.merge(merge, df_slice[i], on=["Location"], suffixes=("", "_%s" % (values[i])))
        
    if len(values) > 2:
        # Fix information for the second group.
        merge["audience_%s" % (values[2])] = merge["audience"]
        del merge["audience"]
    return merge   

def copy_rename(df, oldname, newname):
    return df.rename(columns={oldname: newname}).copy() 

def get_slice(dfin, col, values, frequency="mau"):
    dfout = copy_rename(dfin, '%s_audience' % (frequency), "audience")
    dfout = cut(dfout, col, values, savedcols=['Location', 'audience'])
    dfout["Frequency"] = "Daily" if frequency == "dau" else "Monthly"
    return dfout

def calculate_percentages(df, cols, prefix):
    totals = df.sum(axis=1)
    for col in cols:
        df[prefix + "audience_" + col] = df["audience_" + col] * 100. / totals
        df[prefix + "audience_" + col] = df["audience_" + col] * 100. / totals


In [127]:
#infile = "collections/dataframe_collected_finished_1527597936.csv.gz" # From 29 May
#infile = "collections/dataframe_collected_finished_1528531844.csv.gz" # From 09 Jun
infile = "collections/dataframe_collected_finished_1529526715.csv.gz" # From 20 Jun

df = pd.read_csv(infile)
#df["CollectionDay"] = df["timestamp"].apply(lambda x: convert_unixtime(x))
#print("Collection made in %s" % (df["CollectionDay"].head()[0]))

In [128]:
def extract_relationship(d):
    if 1 in d:
        return "single"
    elif 2 in d:
        return "dating"
    elif 3 in d:
        return "married"
    else:
        return None

def extract_education(d):
    if d == [3, 7, 8, 9, 11]:
        return "graduated"
    elif d == [1, 12, 13]:
        return "no_degree"
    elif d == [2, 4, 5, 6, 10]:
        return "high_school"
    elif d == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        return "all"
    else:
        return None

def extract_group(d):
    id = d[0]["id"]
    if id == 6026404871583:
        return "Expats (Venezuela)"
    elif id == 6015559470583:
        return "Ex-pats (All)"
    else:
        return "All"

def agebuckets(minage, maxage):
    if minage == 13 and (maxage is None or np.isnan(maxage)):
        return "all"
    elif minage == 13 and maxage == 18:
        return "13-18"
    elif minage == 19 and maxage == 30:
        return "19-30"
    elif minage == 31 and maxage == 45:
        return "31-45"
    elif minage == 46 and maxage == 65:
        return "46-65"
    elif minage == 65 and (maxage is None or np.isnan(maxage)):
        return "65+"
    return "undefined"
    
def expand(row):
    place = None
    loc_dimension = None
    if "regions" in row["geo_locations"]:
        place = "%s, %s" % (row["geo_locations"]["regions"][0]["name"], convert_country_code(row["geo_locations"]["regions"][0]["country_code"]))
        loc_dimension = "State"
    elif "countries" in row["geo_locations"]:
        place = convert_country_code(row["geo_locations"]["countries"][0])
        loc_dimension = "Country"
    elif "cities" in row["geo_locations"]:
        place = "%s, %s, %s" % (row["geo_locations"]["cities"][0]["name"], row["geo_locations"]["cities"][0]["region"], convert_country_code(row["geo_locations"]["cities"][0]["country"]))
        loc_dimension = "City"
    
    loctype = "_".join(row["geo_locations"]["location_types"])

    relationship, education, group = None, None, None
    for dimension in row["flexible_spec"]:
        if "relationship_statuses" in dimension:
            relationship = extract_relationship(dimension["relationship_statuses"])
        elif "education_statuses" in dimension:
            education = extract_education(dimension["education_statuses"])
        elif "behaviors" in dimension:
            group = extract_group(dimension["behaviors"])
    
    gender = row["genders"][0]
    gender = "both" if gender == 0 else "man" if gender == 1 else "woman"
    
    return row["age_min"], row["age_max"], place, loc_dimension, loctype, gender, relationship, education, group

def get_item(x):
    if not x:
        return None
    return x["name"]

df[["MinAge","MaxAge","Location","LocationHierarchy","LocationType","Gender","Relationship","Education","Group"]] = df["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)

if "citizenship" in df:
    df["Group"] = df["citizenship"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
    print("Updating Group information...")
    
if "access_device" in df:
    df["Device"] = df["access_device"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
    print("Adding information regarding devices...")
else:
    df["Device"] = None
    
df["AgeBucket"] = df[["MinAge","MaxAge"]].apply(lambda x: agebuckets(x["MinAge"], x["MaxAge"]), axis=1)

# Simple way to collect people's interest. Need a better way in case more than one politician is queries.
df["Politician"] = df["interests"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))

print("Removing redundant cols")

for col in ["Unnamed: 0", "all_fields", "targeting", 'behavior', 'citizenship', "mock_response", "access_device", 
            "ages_ranges", "household_composition", 'interests', 'family_statuses', 'genders', 'geo_locations', 'languages',
            'name', 'relationship_statuses', 'response', 'scholarities', 'timestamp', 'publisher_platforms',]:
    if col in df.keys():
        del df[col]


df["Politician"] = df["Politician"].replace("Luiz Inácio Lula da Silva", "Lula").replace("Marina Silva","Marina").\
                replace("Jair Bolsonaro", "Bolsonaro").replace("Ciro Gomes","Ciro").replace("Geraldo Alckmin", "Alckmin").\
                replace("Alvaro Dias", "Dias").replace("Brazilian Democratic Movement Party", "PMDB").replace("Workers' Party (Brazil)","PT").\
                replace('psdb',"PSDB")
    
print("All Done!")

politicians = ["Lula","Marina","Bolsonaro","Ciro", "Alckmin", "Dias"]
age = ["13-18","19-30","31-45","46-65","65+"]
# Brief description: 
# -----------------
# Gender: 0 Both, 1 Man, 2 Woman
# Age: 13-Null, 13-18, 19-25, 26-40, 40,65, 65+
# LocationType: home_recent, home, recent


Removing redundant cols
All Done!


In [269]:
dfpolitician = {}

dfcut = df[(~df["Politician"].isnull()) & (df["Education"].isnull()) & (df["Device"].isnull()) & (df["Relationship"].isnull()) & (df["AgeBucket"] == "all") & (df["Gender"] == "both")].copy()

dfpolitician["mau"] = get_slice(dfcut, "Politician", politicians, frequency="mau")
dfpolitician["dau"] = get_slice(dfcut, "Politician", politicians, frequency="dau")

calculate_percentages(dfpolitician["mau"], politicians, "%")
calculate_percentages(dfpolitician["dau"], politicians, "%")


In [270]:
dfage = {}
dfcut = df[(df["Education"].isnull()) & (df["Device"].isnull()) & (df["Relationship"].isnull()) & (df["AgeBucket"].apply(lambda x : x in ages)) &
    (df["Politician"].isnull()) & (df["Gender"] == "both") ].copy()
dfage["mau"] = get_slice(dfcut, "AgeBucket", ages, frequency="mau") # Adult is missing...need to check why.
dfage["dau"] = get_slice(dfcut, "AgeBucket", ages, frequency="dau")

In [271]:
# Create age pyramid per gender
# Colums are names audience_AGE_POLITICIAN_[man|woman]

df_age_politician = {}
dfcut = {}

for version in ["mau_man", "dau_man", "dau_woman", "mau_woman"]:
    df_age_politician[version] = {}

for politician in politicians:
    
    for gender in ["man", "woman"]:
        dfcut[gender] = {}
        
        dfcut[gender][politician] = df[(df["Politician"] == politician) & (df["AgeBucket"].apply(lambda x : x in ages)) & (df["Gender"] == gender) & 
                               (df["Education"].isnull()) & (df["Device"].isnull()) & (df["Relationship"].isnull())]

        df_age_politician["mau_" + gender][politician] = get_slice(dfcut[gender][politician], "AgeBucket", ages, frequency="mau")
        df_age_politician["dau_" + gender][politician] = get_slice(dfcut[gender][politician], "AgeBucket", ages, frequency="dau")
    
    for version in ["mau_man", "dau_man", "dau_woman", "mau_woman"]:
        for col in df_age_politician[version][politician].keys():
            if col.startswith("audience"):
                df_age_politician[version][politician][col + "_" + politician] = df_age_politician[version][politician][col]
                del df_age_politician[version][politician][col]

for variation in ["dau","mau"]:
    for gender in ["man", "woman"]:
        key = "tmp_" + variation + "_" + gender
        df_age_politician[key] = pd.merge(df_age_politician[variation + "_" + gender][politicians[0]], df_age_politician[variation + "_" + gender][politicians[1]], on=["Location","Frequency"])
        for i in range(2, len(politicians)):
            df_age_politician[key] = pd.merge(df_age_politician[key], df_age_politician[variation + "_man"][politicians[i]], on=["Location","Frequency"])

df_age_politician["mau"] = pd.merge(df_age_politician["tmp_mau_man"], df_age_politician["tmp_mau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))
df_age_politician["dau"] = pd.merge(df_age_politician["tmp_dau_man"], df_age_politician["tmp_dau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))


In [307]:
# Create age pyramid per gender
# Colums are names audience_POLITICIAN_AGE_[man|woman]
# TODO: names are terrible. Needs improvement...

df_politician_gender = {}
dfcut = {}

for version in ["mau_man", "dau_man", "dau_woman", "mau_woman"]:
    df_politician_gender[version] = {}

for age in ages:
    
    for gender in ["man", "woman"]:
        dfcut[gender] = {}
        
        dfcut[gender][age] = df[(~df["Politician"].isnull()) & (df["AgeBucket"] == "all") & (df["Gender"] == gender) & 
                               (df["Education"].isnull()) & (df["Device"].isnull()) & (df["Relationship"].isnull())]

        df_politician_gender["mau_" + gender][age] = get_slice(dfcut[gender][age], "Politician", politicians, frequency="mau")
        df_politician_gender["dau_" + gender][age] = get_slice(dfcut[gender][age], "Politician", politicians, frequency="dau")
    
    for version in ["mau_man", "dau_man", "dau_woman", "mau_woman"]:
        for col in df_politician_gender[version][age].keys():
            if col.startswith("audience"):
                df_politician_gender[version][age][col + "_" + age] = df_politician_gender[version][age][col]
                del df_politician_gender[version][age][col]


for variation in ["dau","mau"]:
    for gender in ["man", "woman"]:
        key = "tmp_" + variation + "_" + gender
        df_politician_gender[key] = pd.merge(df_politician_gender[variation + "_" + gender][ages[0]], df_politician_gender[variation + "_" + gender][ages[1]], on=["Location","Frequency"])
        for i in range(2, len(ages)):
            df_politician_gender[key] = pd.merge(df_politician_gender[key], df_politician_gender[variation + "_man"][ages[i]], on=["Location","Frequency"])

df_politician_gender["mau"] = pd.merge(df_politician_gender["tmp_mau_man"], df_politician_gender["tmp_mau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))
df_politician_gender["dau"] = pd.merge(df_politician_gender["tmp_dau_man"], df_politician_gender["tmp_dau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))


In [273]:
for dfopt in [dfpolitician, dfage, df_age_politician]:
    dfopt["dau"] = dfopt["dau"].drop_duplicates()
    dfopt["mau"] = dfopt["mau"].drop_duplicates()

In [286]:
merged = {}

merged["mau"] = pd.merge(dfpolitician["mau"], dfage["mau"]).merge(df_age_politician["mau"])
merged["dau"] = pd.merge(dfpolitician["dau"], dfage["dau"]).merge(df_age_politician["dau"])

concated = pd.concat([merged["mau"], merged["dau"]])

allpopulation = df[(df["Politician"].isnull()) & (df["Education"].isnull()) & (df["Device"].isnull()) & (df["Relationship"].isnull()) & (df["AgeBucket"] == "all") & (df["Gender"] == "both")].copy()

dffinal = pd.merge(allpopulation[["dau_audience","mau_audience","Location","LocationHierarchy"]], concated)
dffinal["TotalPopulation"] = dffinal["mau_audience"].where(dffinal["Frequency"]=="Monthly", dffinal["dau_audience"]) 

dffinal["%population_none"] = 100.
for politician in politicians:
    dffinal["%population_" + politician] = 100.0 * dffinal["audience_" + politician] / dffinal["TotalPopulation"]
    dffinal["%population_none"] = dffinal["%population_none"] - dffinal["%population_" + politician] 

del dffinal["mau_audience"]
del dffinal["dau_audience"]

for col in dffinal.keys():
    if col.startswith("%"):
        print(col)
        dffinal[col] = dffinal[col].round(2)

dffinal.to_csv("june20.csv", index=False)

%audience_Lula
%audience_Marina
%audience_Bolsonaro
%audience_Ciro
%audience_Alckmin
%audience_Dias
%population_none
%population_Lula
%population_Marina
%population_Bolsonaro
%population_Ciro
%population_Alckmin
%population_Dias
